In [1]:
import anndata
import os
import numpy as np
import k3d
import scipy
import matplotlib as mpl

from k3d.colormaps import matplotlib_color_maps

plot abrabidopsis leaves in 3d，including its stereogram and left view

In [ ]:
dir = 'E:/REGISTRATION_SOFTWARE/algorithm/cell_level_regist/paper/output_data/our_pipeline/abra_leaves_ours/'
ctype_col ='cell_type'
spa_col = 'spatial_elas'

# read in
fli = [int(fname.replace('.h5ad', '')) for fname in os.listdir(dir) if fname.endswith('.h5ad')]
fli.sort()
fli = [str(fname)+'.h5ad' for fname in fli]

spa = np.zeros((1, 3))
cli = []
for i, fname in enumerate(fli):
    adata = anndata.read(os.path.join(dir, fname))
    adata.obsm[spa_col][:,2] = adata.obsm[spa_col][:,2]  * i * 200
    spa = np.concatenate((spa, adata.obsm[spa_col]), axis=0)
    # keep the same rule of ctype_all generation, as when anncell_color (dic) is generated, in visualization.py, so we can skip the legend generation in k3d, also keep the consistancy of color assignment
    cli += adata.obs[ctype_col].tolist()
    
spa = spa[ 1:, :]


anno_cell_li = list(dict.fromkeys(cli))
anncell_cid = dict(zip(anno_cell_li, range(len(anno_cell_li))))

In [ ]:
anncell_cid

In [ ]:
plt_points = k3d.points(
                                            positions=spa, 
                                            point_size=50, 
                                            shader='3d', 
                                            opacity=1, 
                                            color_map = matplotlib_color_maps.Rainbow,
                                            attribute=[anncell_cid[ele] for ele in cli]
                                        )  # color_range用于展示特定细胞类型
plot = k3d.plot()
plot += plt_points
plot.display()

plot left view of drosophila embryo, before and after elastic registration, maybe its mark genes

In [14]:
dir = 'E:/REGISTRATION_SOFTWARE/algorithm/cell_level_regist/paper/output_data/our_pipeline/drosophila_embryo_ours'
ctype_col = 'annotation'
spa_col = 'spatial_elas'
gene_name_li = ['Rbp6'] #['Cpr56F']# ['Osi7']   #  # 

# read in
fli = [int(fname.replace('.h5ad', '')) for fname in os.listdir(dir) if fname.endswith('.h5ad')]
fli.sort()
fli = [str(fname)+'.h5ad' for fname in fli]

spa = np.zeros((1, 3))
cli = []
gene_sum = []
for fname in fli:
    adata = anndata.read(os.path.join(dir, fname))
    spa = np.concatenate((spa, adata.obsm[spa_col]), axis=0)
    # keep the same rule of ctype_all generation, as when anncell_color (dic) is generated, in visualization.py, so we can skip the legend generation in k3d, also keep the consistancy of color assignment
    cli += adata.obs[ctype_col].tolist()
    
    # summarize expression count of certain gene(s)
    adata_cp = anndata.AnnData(X=adata.X, var=adata.var)
    adata_cp.X = adata.layers['raw_counts']
    if scipy.sparse.issparse(adata_cp.X):
        adata_cp.X = adata_cp.X.todense()
    gene_sum += list(adata_cp[:, gene_name_li].X.sum(axis=1))
spa = spa[ 1:, :]

anno_cell_li = list(dict.fromkeys(cli))
anncell_cid = dict(zip(anno_cell_li, range(len(anno_cell_li))))

In [ ]:
set(gene_sum)

In [13]:
# plot cell type
plt_points = k3d.points(
                        positions=spa, 
                        point_size=0.3, 
                        shader='3d', 
                        opacity=1, 
                        color_map = matplotlib_color_maps.Rainbow,
                        attribute=[anncell_cid[ele] for ele in cli]
                    )  # color_range用于展示特定细胞类型
plot = k3d.plot()
plot += plt_points
plot.display()

D:\anaconda\envs\k3d\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [17]:
ind_ept = np.where(np.array(gene_sum) < 3)  # 3
ind_exp = np.where(np.array(gene_sum) >=3)
attr_val = np.array(gene_sum)[ind_exp]  # np.log10

plt_points_exp = k3d.points(
                        positions=spa[ind_exp, :], 
                        point_size=0.3, 
                        shader='flat', #'3d', 
                        opacity=1, 
                        color_map = matplotlib_color_maps.Purples, #GnBu, # Blues # Purples
                        attribute=attr_val, #[anncell_cid[ele] for ele in cli]
                        color_range=[-20, attr_val.max()]  # -20, -15
                    )  # color_range用于展示特定细胞类型

plt_points_ept = k3d.points(
                        positions=spa[ind_ept, :], 
                        point_size=0.05, 
                        shader='flat', #'3d', 
                        opacity=1, 
                        color= 0xffffff)    # 0xDCDCDC

plot = k3d.plot() 
plot += plt_points_exp
plot += plt_points_ept
plot.display()

Output()

plot left view of drosophila larva, before and after elastic registration, maybe its mark genes

In [10]:
dir = 'E:/REGISTRATION_SOFTWARE/algorithm/cell_level_regist/paper/output_data/our_pipeline/drosophila_larva_ours'
ctype_col = 'annotation'
spa_col = 'spatial_elas'
# read in
fli = [int(fname.replace('.h5ad', '')) for fname in os.listdir(dir) if fname.endswith('.h5ad')]
fli.sort()
fli = [str(fname)+'.h5ad' for fname in fli]

spa = np.zeros((1, 3))
cli = []
for i, fname in enumerate(fli):
    adata = anndata.read(os.path.join(dir, fname))
    
     # revise z coor for corrected visualization
    cor_slice = adata.obsm[spa_col]
    cor_slice[:, 2] = i * 2
    
    spa = np.concatenate((spa, cor_slice), axis=0)
    
    # keep the same rule of ctype_all generation, as when anncell_color (dic) is generated, in visualization.py, so we can skip the legend generation in k3d, also keep the consistancy of color assignment
    cli += adata.obs[ctype_col].tolist()
spa = spa[ 1:, :]

anno_cell_li = list(dict.fromkeys(cli))
# anncell_cid = dict(zip(anno_cell_li, range(len(anno_cell_li))))

np.random.seed(10)  # seed_val when generating 2d plot
color_arr = mpl.colors.ListedColormap(np.random.rand(len(anno_cell_li), 3)).colors

attr_arr = np.arange(len(anno_cell_li))/(len(anno_cell_li)-1)

ctype_attr = dict(zip(anno_cell_li, attr_arr))
color_map_k3d = np.concatenate([np.expand_dims(attr_arr , axis=1), color_arr], axis=1)

In [ ]:
# x,y  increment = 2.5  -> 25μm (bin50)| 1 -> 10μm
# z increment = 20μm -> 2 (bin40)


In [11]:
plt_points = k3d.points(
                                            positions=spa, 
                                            point_size=1, 
                                            shader='3d', 
                                            opacity=1, 
                                            color_map = color_map_k3d,  # matplotlib_color_maps.Rainbow,
                                            attribute=[ctype_attr[ele] for ele in cli]
                                        )  # color_range用于展示特定细胞类型
plot = k3d.plot()
plot += plt_points
plot.display()

D:\anaconda\envs\k3d\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

plot mouse embryo in 3d，including its perspective, main view, left view and top view, then stereogram with its mesh

In [ ]:
dir = 'E:/REGISTRATION_SOFTWARE/algorithm/cell_level_regist/paper/output_data/our_pipeline/mouse_embryo_E11.5_34_70_ours'
ctype_col = 'annotation'

# read in
fli = [int(fname.replace('.h5ad', '')) for fname in os.listdir(dir) if fname.endswith('.h5ad')]
fli.sort()
fli = [str(fname)+'.h5ad' for fname in fli]

spa = np.zeros((1, 3))
cli = []
for i, fname in enumerate(fli[8:]):  
    adata = anndata.read(os.path.join(dir, fname))
    
     # revise z coor for corrected visualization, pinned to the false z value generated in main_gen_data_mouse_embryo_E11.5.py
    cor_slice = adata.obsm['spatial_elas']
    cor_slice[:, 2] *= 2
    
    spa = np.concatenate((spa, cor_slice), axis=0)
    
    # keep the same rule of ctype_all generation, as when anncell_color (dic) is generated, in visualization.py, so we can skip the legend generation in k3d, also keep the consistancy of color assignment
    cli += adata.obs[ctype_col].tolist()
spa = spa[ 1:, :]

anno_cell_li = list(dict.fromkeys(cli))
np.random.seed(14)
color_arr = mpl.colors.ListedColormap(np.random.rand(len(anno_cell_li), 3)).colors

attr_arr = np.arange(len(anno_cell_li))/(len(anno_cell_li)-1)

ctype_attr = dict(zip(anno_cell_li, attr_arr))
color_map_k3d = np.concatenate([np.expand_dims(attr_arr , axis=1), color_arr], axis=1)

# color_li = cm.rainbow(np.linspace(0, 1, len(anno_cell_li)))
# anncell_color = dict(zip(anno_cell_li, color_arr))

In [ ]:
plt_points = k3d.points(
                                            positions=spa, 
                                            point_size=50, 
                                            shader='3d', 
                                            opacity=1, 
                                            color_map = color_map_k3d,
                                            attribute=[ctype_attr[ele] for ele in cli]
                                        )  # color_range用于展示特定细胞类型
plot = k3d.plot(grid=(-6000, -6000, 0, 6000, 6000, 4000))
plot += plt_points
plot.display()

plot mouse brain in 3d

In [ ]:
def rgb_to_hex(rgb):
    return  int('%02x%02x%02x' % rgb, 16)

dir = 'E:/REGISTRATION_SOFTWARE/algorithm/cell_level_regist/paper/output_data/our_pipeline/mouse_brain_ours_sigma0.3/'
ctype_col ='annotation'
spa_col = 'spatial_elas'

# read in
fli = [int(fname.replace('.h5ad', '')) for fname in os.listdir(dir) if fname.endswith('.h5ad')]
fli.sort()
fli = [str(fname)+'.h5ad' for fname in fli]

spa = np.zeros((1, 3))
cli = []
for i, fname in enumerate(fli):
    adata = anndata.read(os.path.join(dir, fname))
    adata.obsm[spa_col][:,2] = adata.obsm[spa_col][:,2]  * i * 200
    spa = np.concatenate((spa, adata.obsm[spa_col]), axis=0)
    # keep the same rule of ctype_all generation, as when anncell_color (dic) is generated, in visualization.py, so we can skip the legend generation in k3d, also keep the consistancy of color assignment
    cli += adata.obs[ctype_col].tolist()
    
spa = spa[ 1:, :]

anno_cell_li = list(dict.fromkeys(cli))
seed_val=11
np.random.seed(seed_val)
color_arr = mpl.colors.ListedColormap(np.random.rand(len(anno_cell_li), 3)).colors * 255
color_arr = color_arr.astype(int)

anncell_hex = dict(zip(anno_cell_li, [rgb_to_hex(tuple(rgb)) for rgb in color_arr]))

In [ ]:
anncell_hex

In [ ]:
plt_points = k3d.points(
                                            positions=spa, 
                                            colors=np.array([anncell_hex[ele] for ele in cli]),
                                            point_size=50, 
                                            shader='3d', 
                                            opacity=1, 
                                            # color_map = matplotlib_color_maps.Rainbow,
                                            # attribute=[anncell_cid[ele] for ele in cli]
                                        )  # color_range用于展示特定细胞类型
plot = k3d.plot()
plot += plt_points
plot.display()

In [ ]:
anncell_hex

In [ ]:
celltype = 'L6b'

c_arr  = np.array(cli)

in_ind = np.where(np.array(c_arr)  == celltype) 
ex_ind = np.where(np.array(c_arr)  != celltype) 

plt_points_exp = k3d.points(
                                                    positions=spa[in_ind, :], 
                                                    colors=np.array([anncell_hex[celltype]] * in_ind[0].shape[0]),
                                                    point_size=50, 
                                                    shader='flat', #'3d', 
                                                    opacity=1
                                                )  # color_range用于展示特定细胞类型

plt_points_ept = k3d.points(
                        positions=spa[ex_ind, :], 
                        point_size=0.05, 
                        shader='flat', #'3d', 
                        opacity=1, 
                        color= 0xffffff)    # 0xDCDCDC

plot = k3d.plot() 
plot += plt_points_exp
plot += plt_points_ept
plot.display()

In [ ]:
in_ind